Second Phase Environment implemented based on Env_P1 but for more herds in a population

In [1]:
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import copy
import random
from scipy.stats import poisson
from scipy.stats import geom
from scipy.stats import hypergeom

from tf_agents.environments import py_environment
from tf_agents.specs import BoundedArraySpec
from tf_agents.trajectories.time_step import StepType
from tf_agents.trajectories.time_step import TimeStep

# Documentation
Env_P2_N is a class that represents an epidemic with n herds.  

<img src="Sketch_P2.jpeg"
     alt="Env_P2_N Sketch"
     style="float: left; margin-right: 5px;" />
## Variables:  
The action $\in \mathbb{R}^4$ is a vector $(\tau_1, \tau_2, s_1, s_2)$.  
$\tau_i$ are the number of tests to be done in herd $i$.  
$s_i \in \{0,1\}$ determine whether a herd is to be completely replaced by healthy members.  
$S_i$ is the number of susceptible herd members (subjects) of herd $i$, $i \in {0,1}$.  
$I_i$ is the number of infectious subjects of herd $i$, $i \in {1,2}$.  
$g$ is a small recovery probability.  
$q$ is a small infection probability.  
$B_i' = min(B_i,|S_i|-1)$ with $B_i = \sum_{I_i} A$ and $A \sim Poi(0.01)$, is the number of Susceptibles moving to Infectious per time step for herd $i$, $i \in {1,2}$.  
## State  
The state contains two arrays of size six. 
state[0] is the observation shown to the agent.  
The observation are testresults for each herd $(\mu_i, x^i_0, x^i_1)$.  
$\mu_i$ shows the number of time steps since the test has been done.  
$x^i_0$ and $x^i_1$ correspond to negative and positive testresults respectively.  

state[1] instead contains the internal information about both herds.  
state[1][i] shows the population size for herd i.  
state[1][i+2] shows the total number of infected subjects for herd i.  
## Reward  
Reward calculation respects costs for tests and herd replacement.  
Let $R$ be the current reward (starts at zero). Then each time step:  

$R - \tau_i * c + 1_{\tau_i} * cprime, \forall i \in \{0,1\}$,  
$R - s_i * state[1][i] * e, \forall i \in \{0,1\}$. 
  
Here, $c$, $cprime > c$ and $e$ are constants.  
Also, reduces reward by number of infectious:  

$R - state[1][i+2], \forall i \in \{0,1\}$. 


In [2]:
class Env_P2_N(py_environment.PyEnvironment):
    def __init__(self,
                num_herds = 10,
                total_population = 3000,
                split_even = True,
                population_range = None,
                num_transfers = 10,
                weeks_until_exchange = 4,
                rand_recovery_prob = 0.005,
                rand_infection_prob = 0.01
                ):
        super(Env_P2_N, self).__init__()
        self._state = np.zeros(((num_herds*2),), np.int32)
        self._observation = np.zeros(((num_herds*3),), np.int32)
        self._discount = np.float32(1)
        self._time = 0
        self._episode_length = 0
        self._tests = []
        self._reward = np.float32(0)
        self._c_tests = 0.5   #cost for each test
        self._c_prime_tests = 10    #organizational costs tests
        self._e_removed = 3   #individual replacement cost
        self._weeks_until_testresults = 3
        self._split_even = split_even
        self._population_range = population_range
        self._num_herds = num_herds
        self._num_transfers = num_transfers
        self._total_population = total_population
        self._exchanged_members = np.int32(np.round(total_population / (num_transfers*num_herds*5)))    #k from scrapsheet
        self._weeks_until_exchange = weeks_until_exchange    #T from scrapsheet
        self._rand_recovery_prob = rand_recovery_prob    #g from scrapsheet
        self._rand_infection_prob = rand_infection_prob    #q from scrapsheet
    
    def action_spec(self):
        #Actions for: number of subjects to be tested h1, h2. number of subjects to be eliminated h1, h2
        max_array = np.ones(((self._num_herds*2),), np.int32)
        for i in range (0, self._num_herds):
            max_array[i] = self._state[i]
        return BoundedArraySpec(((self._num_herds*2),), np.int32, minimum=0, maximum=max_array)
    
    
    def observation_spec(self):
        # tau, x0, x1 for both herds
        return BoundedArraySpec(((self._num_herds*3),), np.int32, minimum=0, maximum=array(self._total_population))
    
    
    def _reset(self):
        '''
        State consists of actual state of each herd (population and infected, state[1]),
        and observation the agent gets to see (state[0]).
        state[0] contains:
        number of steps since test has taken place,
        number of positive tests,
        number of negative tests
        for each herd.  
        '''
        self._state = np.zeros(((self._num_herds*2),), np.int32)
        if self._split_even:
            for i in range (0, self._num_herds):
                self._state[i] = self._total_population / self._num_herds
        else:
            raise NameError('Work more Maurice.')
        
        initial_infected_h1 = np.random.randint(low = 1, high = (self._state[0]/8))
        self._tests = []
        self._time = 0
        self._reward = np.float32(0)
        self._episode_length = geom.rvs(p = 1/270)
        self._state[self._num_herds] = initial_infected_h1    #infected h1
        self._observation = np.zeros(((self._num_herds*3),), np.int32)
        return TimeStep(StepType.FIRST, reward=self._reward,
                    discount=self._discount, observation = self._observation)
    
    def _test(self, herd = -1, num_tests = 0):
        '''
        Randomly draws (without returning) num_tests subjects of a herd,
        then tests whether they are infected or not before returning testresults.
        '''
        assert self._state[herd] >= num_tests, "More tests than herd members."
        if herd >= 0 and num_tests > 0:
            test_out = hypergeom.rvs(M = self._state[herd], n = self._state[herd+self._num_herds], N = int(num_tests), size = None)
            testresults = np.zeros(3, np.int32)
            testresults[1] = num_tests - test_out #negative tests
            testresults[2] = test_out #positive tests
            return testresults
        else:
            return np.zeros(3, np.int32)
        
    def _transfer(self, origin_herd = -1, target_herd = -1):
        ''' 
        Each self._weeks_until_exchange weeks, transfers subjects 
        from origin_herd to target_herd by randomly drawing (without return)
        self._exchanged_members subjects from all subjects of origin_herd.
        returns numbers of infected transfers and susceptible transfers.
        '''

        assert self._state[origin_herd] > self._exchanged_members, "Population in origin herd too low."
        if origin_herd >= 0 and target_herd >=0 and self._time % self._weeks_until_exchange == 0:
            infected_transfers = hypergeom.rvs(M = self._state[origin_herd], 
                                                 n = self._state[origin_herd+self._num_herds], N = self._exchanged_members, size = None)
            susceptible_transfers = self._exchanged_members - infected_transfers
            return np.array([susceptible_transfers, infected_transfers])    
        else:
            return None
        
    def _model(self, action: np.ndarray):
        '''
        Completes one time step in a herd (i.e. excluding transfers and tests).
        In f(x), samples new infections from poisson dist with lambda = 0.05,
        also considers spontaneous infection and recovery factors.
        Then, depending on whether a herd is to be replaced by healthy subjects (action),
        calls f(x) or simply replaces all subjects by healthy subjects for each herd.
        '''
        #Model for one herd
        def f(x):
            S = np.int32(x[0])
            I = np.int32(x[1])
            new_infs = 0
            inf_rvs = poisson.rvs(0.05, size = I)
            rec_rvs = poisson.rvs(self._rand_recovery_prob, size = I)
            rand_rvs = poisson.rvs(self._rand_infection_prob, size = S)
            potential_infs = np.sum(inf_rvs)
            recoveries = np.sum(rec_rvs)
            rand_infs = np.sum(rand_rvs)
            potential_infs = min(potential_infs, S)
            #code draw whether subject to be infected is already infected or sus
            if potential_infs >= 1:
                new_infs = hypergeom.rvs(M = (S + I), n = S, N = potential_infs, size = None)
            new_I = I + new_infs + rand_infs - recoveries
            return new_I
        
        #One step for each herd
        step_results = self._state
        temp_x = np.zeros((2,), np.int32)
        for i in range (self._num_herds, self._num_herds*2):
            if action[i] == 1:
                step_results[i] = 0
            else:
                step_results[i] = f(np.array([(self._state[i-self._num_herds] - self._state[i]), self._state[i]]))
        return step_results
    
    def _reward_func(self, action: np.ndarray):
        '''
        Calculates and returns reward.
        '''
        for i in range (0, self._num_herds):
            self._reward -= self._discount * (action[i] * self._c_tests + min(action[i],1) * self._c_prime_tests) / (self._total_population/self._num_herds)
            self._reward -= self._discount * (action[i+self._num_herds] * self._state[i] * self._e_removed ) / (self._total_population/self._num_herds)
            self._reward -= self._discount * (self._state[i+self._num_herds])**1.5 / (self._total_population/self._num_herds)
        return self._reward
    
    def _step(self, action: np.ndarray):
        '''
        Step completes one time step in the environment.
        First, transfers subjects if time interval dictates it.
        Then, calls model(action) to complete a time step in each herd.
        Afterwards, tests subjects if action dictates it and outputs testresults
        if time for testing has been concluded.
        Finally, calculates reward and returns a Time_Step object.
        TimeStep(StepType.MID, reward=reward, discount=self._discount, observation=self._observation)
        '''
        if self._current_time_step.is_last():
            return self.reset()
            
        self._time += 1
        
        #Transfers
        indices = np.arange(self._num_herds)
        np.random.shuffle(indices)
        for i in range (0, self._num_transfers):
            origin_herd = indices[i % (self._num_herds-1)]
            if i % self._num_herds == 0:
                target_herd = indices[self._num_herds-1]
            else:
                target_herd = indices[(i % (self._num_herds-1))-1]
            transfers = self._transfer(origin_herd = origin_herd, target_herd = target_herd)
            back_transfers = self._transfer(origin_herd = target_herd, target_herd = origin_herd)
            if transfers is not None:
                self._state[origin_herd+self._num_herds] = self._state[origin_herd+self._num_herds] - transfers[1] + back_transfers[1]
                self._state[target_herd+self._num_herds] = self._state[target_herd+self._num_herds] + transfers[1] - back_transfers[1]
            
        #Model should make a step in between transfer and test
        self._state = self._model(action)
                
        #Testing
        for i in range (0, self._num_herds):
            self._tests.append(self._test(herd = i, num_tests = action[i]))
        lim = np.ma.size(self._tests, axis = 0)
        get_obs = False
        

        for i in reversed (range (0, lim)):
            if self._tests[i][0] < self._weeks_until_testresults:
                self._tests[i][0] += 1
            elif self._tests[i][0] > self._weeks_until_testresults:
                raise ValueError()
            elif self._tests[i][0] == self._weeks_until_testresults:
                get_obs = True
                break
        
        if get_obs:
            for j in range (0, self._num_herds):
                k = j*3
                if self._tests[j][1] == 0 and self._tests[j][2] == 0:
                    self._observation[k] += 1
                else:
                    self._observation[k] = self._tests[j][0]
                    self._observation[k+1] = self._tests[j][1]
                    self._observation[k+2] = self._tests[j][2]

            for k in range(0, self._num_herds):
                assert self._tests[k][0] == self._weeks_until_testresults, "A Test that should not have been removed has been removed."
                self._tests.pop(k)
        else:
            for l in range (0, np.size(self._observation), 3):
                self._observation[l] += 1
        

        #Reward function
        self._reward = self._reward_func(action)
        step_reward = np.float32(0)
        
        #Debugging
        #if self._time % 50 == 0:
            #print('Hidden State: ', self._state)
            #print('Observation: ', self._observation)
            
        #output
        if self._time == self._episode_length:
            return TimeStep(StepType.LAST, reward=self._reward, discount=self._discount, observation=self._observation)
            
        else:
            return TimeStep(StepType.MID, reward=step_reward, discount=self._discount, observation=self._observation)

In [3]:
'''Testing environment.'''
n = 4
pop = 30000
env = Env_P2_N(num_herds = n, total_population = pop)
action = np.zeros((2*n,), np.int32)


time_step = env.reset()
aspec = env.action_spec()
counter = 0
i = 0
print("Random policy.")
while True:
    if (counter == 5):
        counter = 0
        i = 0
        print("---------------------------------------------")
        break
    for j in range (0, (2*n)):
        action[j] = np.random.randint(low = aspec.minimum, high = aspec.maximum[j])    
    time_step = env.step(action)
    i +=1
    if env._current_time_step.is_last():
        counter += 1
        print("Final Reward after ", i , " Steps: ", time_step.reward)
        i = 0


action = np.zeros((2*n,), np.int32)
time_step = env.reset()
aspec = env.action_spec()
counter = 0
i = 0
print("Test 100%, replace when more than 50% infected.")
while True:
    if (counter == 5):
        counter = 0
        i = 0
        print("---------------------------------------------")
        break
    for j in range (0, (2*n)):
        if j < n:
            action[j] = aspec.maximum[j]
        elif j >= n: 
            if time_step.observation[((j-n)*3)+2] > time_step.observation[((j-n)*3)+1]:
                action[j] = 1
            else:
                action[j] = 0
        else:
            print("how did this happen?")  
    time_step = env.step(action)
    i +=1
    if env._current_time_step.is_last():
        counter += 1
        print("Final Reward after ", i , " Steps: ", time_step.reward)
        i = 0
        

action = np.zeros((2*n,), np.int32)        
time_step = env.reset()
aspec = env.action_spec()
counter = 0
i = 0
print("Test 50%, replace when more than 50% infected.")
while True:
    if (counter == 5):
        counter = 0
        i = 0
        print("---------------------------------------------")
        break
    for j in range (0, (2*n)):
        if j < n:
            action[j] = aspec.maximum[j]/2
        elif j >= n: 
            if time_step.observation[((j-n)*3)+2] > time_step.observation[((j-n)*3)+1]:
                action[j] = 1
            else:
                action[j] = 0
        else:
            print("how did this happen?")  
    time_step = env.step(action)
    i +=1
    if env._current_time_step.is_last():
        counter += 1
        print("Final Reward after ", i , " Steps: ", time_step.reward)
        i = 0


action = np.zeros((2*n,), np.int32)
time_step = env.reset()
aspec = env.action_spec()
counter = 0
i = 0
print("Do-nothing-policy.")
while True:
    if (counter == 5):
        counter = 0
        i = 0
        print("---------------------------------------------")
        break
    time_step = env.step(action)
    i +=1
    if env._current_time_step.is_last():
        counter += 1
        print("Final Reward after ", i , " Steps: ", time_step.reward)
        i = 0

Random policy.
Final Reward after  96  Steps:  -16565.17407212192
Final Reward after  407  Steps:  -111308.81853656773
Final Reward after  110  Steps:  -20278.455830256884
Final Reward after  315  Steps:  -83385.03456864959
Final Reward after  50  Steps:  -4190.565553871474
---------------------------------------------
Test 100%, replace when more than 50% infected.
Final Reward after  67  Steps:  -3118.681753444821
Final Reward after  56  Steps:  -2535.3854228483788
Final Reward after  87  Steps:  -4845.966196849672
Final Reward after  176  Steps:  -9346.693440899126
Final Reward after  353  Steps:  -18637.952187637326
---------------------------------------------
Test 50%, replace when more than 50% infected.
Final Reward after  23  Steps:  -589.9397514421723
Final Reward after  219  Steps:  -11380.647734022838
Final Reward after  304  Steps:  -16384.110649582457
Final Reward after  1018  Steps:  -52845.772254686
Final Reward after  457  Steps:  -23449.214048744117
------------------